In [395]:
import pandas as pd
import statsmodels.formula.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from xgboost import XGBRegressor
from sklearn import preprocessing
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.special import boxcox,inv_boxcox
import datetime

In [396]:
train=pd.read_csv('train.csv')
train["Cost"]=abs(train["Cost"])
train=train.dropna(subset=['Price Of Sculpture','Material','Base Shipping Price','Weight','Height','Width','Artist Reputation'])
#train=train.replace(to_replace=['Marble','Brass'],value=['Stone','Aluminium'])
train=train.replace(to_replace=['Stone','Clay','Wood','Aluminium','Bronze','Marble','Brass'],value=[1,2,3,4,5,6,7])
#train['Weight']=train['Weight'].fillna(0)
#train['Width']=train['Width'].fillna(0)
#train['Height']=train['Height'].fillna(0)
#train['Artist Reputation']=train['Artist Reputation'].fillna(0)

del train["Artist Name"]
del train["Customer Location"]
del train["Customer Id"]
del train["International"]
del train["Express Shipment"]
del train["Installation Included"]
del train["Transport"]
del train["Fragile"]
del train["Customer Information"]
del train["Remote Location"]
del train["Scheduled Date"]
del train["Delivery Date"]
train=train.rename(columns={'Artist Reputation':'Artist_Reputation','Price Of Sculpture':'Price_of_Sculpture','Base Shipping Price':'Base_Shipping_Price'})


In [397]:
train['Weight']=boxcox(train['Weight'],0.1)
train['Cost']=boxcox(train['Cost'],0)
#train['Height']=boxcox(train['Height'],0.745)
#train['Width']=boxcox(train['Width'],0.635)
train['Artist_Reputation']=boxcox(train['Artist_Reputation'],0.86)
train['Price_of_Sculpture']=boxcox(train['Price_of_Sculpture'],-0.70)
train['Base_Shipping_Price']=boxcox(train['Base_Shipping_Price'],-0.48)
pd.Series(train['Cost']).skew()

1.7592747703196607

In [398]:
y=train['Cost']
del train['Cost']

In [399]:
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3,random_state=123)
#X_train=X_train.assign(Cost=y_train)

In [400]:
#mdl=sm.ols("Cost ~ (Price_of_Sculpture+Base_Shipping_Price)*Material*Height*Width*Weight*Artist_Reputation",data=X_train).fit()

In [401]:
weights=[1,1,1,1,1,1.0]

In [402]:
mdl = XGBRegressor(n_estimators=1000,objective='reg:squarederror',min_child_weight=1)

mdl.fit(X_train,y_train,feature_weights=weights)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [403]:
pred=mdl.predict(X_test)
pred=inv_boxcox(pred,0)
y_test=inv_boxcox(y_test,0)
print(metrics.r2_score(y_test,pred))

0.6738488230570336


In [404]:
score=100*max(0, 1-metrics.mean_squared_log_error(y_test, pred))

In [405]:
print(score)

91.45028428427528


In [409]:
test=pd.read_csv('test.csv')
del test["Artist Name"]
del test["Customer Location"]
del test["Customer Id"]
del test["International"]
del test["Express Shipment"]
del test["Installation Included"]
del test["Transport"]
del test["Fragile"]
del test["Customer Information"]
del test["Remote Location"]
del test["Scheduled Date"]
del test["Delivery Date"]
#test['Weight']=test['Weight'].fillna(1)
#test['Width']=test['Width'].fillna(1)
#test['Height']=test['Height'].fillna(1)
#test['Artist Reputation']=test['Artist Reputation'].fillna(0)
test=test.rename(columns={'Artist Reputation':'Artist_Reputation','Price Of Sculpture':'Price_of_Sculpture','Base Shipping Price':'Base_Shipping_Price'})
test=test.replace(to_replace=['Marble'],value='Stone')
test=test.replace(to_replace=['Stone','Clay','Wood','Aluminium','Brass','Bronze'],value=[1,2,3,4,5,6])
test['Weight']=boxcox(test['Weight'],0.1)
test['Height']=boxcox(test['Height'],0.745)
test['Width']=boxcox(test['Width'],0.635)
test['Artist_Reputation']=boxcox(test['Artist_Reputation'],0.86)
test['Price_of_Sculpture']=boxcox(test['Price_of_Sculpture'],-0.70)
test['Base_Shipping_Price']=boxcox(test['Base_Shipping_Price'],-0.48)

In [407]:
p=mdl.predict(test)
p=inv_boxcox(p,0)

In [408]:
test=pd.read_csv('test.csv')
output= pd.DataFrame(columns=['Customer Id', 'Cost'])
output["Customer Id"]=test["Customer Id"]
output["Cost"]=p
output.to_csv("output.csv")